In [33]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._ 
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.split

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.split


In [34]:
val karate_graph = sc.textFile("data/soc-karate/soc-karate.mtx")


karate_graph: org.apache.spark.rdd.RDD[String] = data/soc-karate/soc-karate.mtx MapPartitionsRDD[66] at textFile at <console>:51


In [35]:
var df_karate = karate_graph.toDF.withColumn("id",monotonicallyIncreasingId)


df_karate: org.apache.spark.sql.DataFrame = [value: string, id: bigint]


In [36]:
df_karate = df_karate.withColumn("rank", row_number().over(Window.orderBy("id")))

df_karate: org.apache.spark.sql.DataFrame = [value: string, id: bigint ... 1 more field]


In [37]:
df_karate.show(30,false)

+--------------------------------------------------------------------------------+----------+----+
|value                                                                           |id        |rank|
+--------------------------------------------------------------------------------+----------+----+
|%%MatrixMarket matrix coordinate pattern symmetric                              |0         |1   |
|%-------------------------------------------------------------------------------|1         |2   |
|% UF Sparse Matrix Collection, Tim Davis                                        |2         |3   |
|% http://www.cise.ufl.edu/research/sparse/matrices/Newman/karate                |3         |4   |
|% name: Newman/karate                                                           |4         |5   |
|% [Karate club, from Wayne Zachary, 1977]                                       |5         |6   |
|% id: 2399                                                                      |6         |7   |
|% date: 1

In [38]:
df_karate = df_karate.filter(df_karate("rank")>24)

df_karate: org.apache.spark.sql.DataFrame = [value: string, id: bigint ... 1 more field]


In [39]:
df_karate = df_karate.drop("id","rank")

df_karate: org.apache.spark.sql.DataFrame = [value: string]


In [40]:
val df_karate_splitted = df_karate.withColumn("_tmp", split($"value", "\\ ")).select(
  $"_tmp".getItem(0).as("srcId"),
  $"_tmp".getItem(1).as("dstID"),
)

df_karate_splitted: org.apache.spark.sql.DataFrame = [srcId: string, dstID: string]


In [41]:
val rows: RDD[Row] = df_karate_splitted.rdd

rows: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[84] at rdd at <console>:52


In [42]:
var a= rows.map{ case Row(src:String, dist : String) => Edge(src.toLong, dist.toLong,"friend")}
var default_user=("Default user", "Missing value")
var graph = Graph.fromEdges(a, default_user)

a: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = MapPartitionsRDD[85] at map at <console>:55
default_user: (String, String) = (Default user,Missing value)
graph: org.apache.spark.graphx.Graph[(String, String),String] = org.apache.spark.graphx.impl.GraphImpl@5d5aaccc


In [43]:
graph.edges.collect()

res7: Array[org.apache.spark.graphx.Edge[String]] = Array(Edge(2,1,friend), Edge(3,1,friend), Edge(3,2,friend), Edge(4,1,friend), Edge(4,2,friend), Edge(4,3,friend), Edge(5,1,friend), Edge(6,1,friend), Edge(7,1,friend), Edge(7,5,friend), Edge(7,6,friend), Edge(8,1,friend), Edge(8,2,friend), Edge(8,3,friend), Edge(8,4,friend), Edge(9,1,friend), Edge(9,3,friend), Edge(10,3,friend), Edge(11,1,friend), Edge(11,5,friend), Edge(11,6,friend), Edge(12,1,friend), Edge(13,1,friend), Edge(13,4,friend), Edge(14,1,friend), Edge(14,2,friend), Edge(14,3,friend), Edge(14,4,friend), Edge(17,6,friend), Edge(17,7,friend), Edge(18,1,friend), Edge(18,2,friend), Edge(20,1,friend), Edge(20,2,friend), Edge(22,1,friend), Edge(22,2,friend), Edge(26,24,friend), Edge(26,25,friend), Edge(28,3,friend), Edge(28,24,fr...
